# Ejercicio 7: Bases de Datos Vectoriales

## Darlin Joel Anacicha    Curso: GR1CC

## Objetivo de la práctica

Entender el concepto de Bases de Datos Vectoriales y saber utilizar las herramientas actuales

## Parte 0: Carga del Corpus

Vamos a utilizar la API de Kaggle para acceder al dataset _Wikipedia Text Corpus for NLP and LLM Projects_

El corpus está disponible desde este [link](https://www.kaggle.com/datasets/gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects?utm_source=chatgpt.com)

### Actividad

1. Carga el corpus


In [23]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [24]:
# Set the path to the file you'd like to load
file_path = "wikipedia_text_corpus.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects",
  file_path,
)

df.head()

,Unnamed: 0,text
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...
1,2,Battery indicator\n\nA battery indicator (also...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...


## Parte 1: Generación de Embeddings

Vamos a utilizar E5 como modelo de embeddings.

La documentación de E5 está disponible desde este [link](https://huggingface.co/intfloat/e5-base-v2)

### Actividad

1. Normalizar el corpus
2. Definir una función `chunk_text`, y dividir los textos en _chunks_.
3. Generar embeddings por cada _chunk_

In [25]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

df = df.dropna(subset=["text"]).reset_index(drop=True)

# Limpieza básica
def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].astype(str).map(normalize_text)

df.head()

,Unnamed: 0,text,text_norm
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...,Anovo Anovo (formerly A Novo) is a computer se...
1,2,Battery indicator\n\nA battery indicator (also...,Battery indicator A battery indicator (also kn...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19...","Bob Pease Robert Allen Pease (August 22, 1940Â..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...,CAVNET CAVNET was a secure military forum whic...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...,CLidar The CLidar is a scientific instrument u...


In [26]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    """
    Chunking por caracteres.
    max_chars ~ 600-1000 suele funcionar bien.
    overlap ayuda a no cortar ideas a la mitad.
    """
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end]
        chunk = chunk.strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in df.iterrows():
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": int(i),
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
chunks_df.head(), len(chunks_df)

(   doc_id  chunk_id                                               text
 0       0         0  Anovo Anovo (formerly A Novo) is a computer se...
 1       1         0  Battery indicator A battery indicator (also kn...
 2       1         1  ad battery when in reality it indicates a prob...
 3       1         2  s that an internal standby battery needs repla...
 4       1         3  increase; in many cases the EMF remains more o...,
 79104)

In [27]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"   # recomendado para retrieval
model = SentenceTransformer(MODEL_NAME)

# Textos a indexar (pasajes)
passages = ["passage: " + t for t in chunks_df["text"].tolist()]

In [28]:
# Embeddings (N x D)
# Se debe usar normalize_embeddings=True para similitud coseno
embeddings = model.encode(
    passages,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

Batches:   0%|          | 0/4944 [00:00<?, ?it/s]

In [29]:
print(embeddings.shape, embeddings.dtype)

(79104, 768) float32


In [30]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

query_text = "Battery measuring"

query_vec = embed_query(query_text)
query_vec.shape

(1, 768)

## Parte 2: FAISS

FAISS es una librería para búsqueda por similitud eficiente y clustering de vectores densos.

La documentación de FAISS está disponible en este [link](https://faiss.ai/index.html)

### Actividad

1. Crea un índice en FAISS
2. Carga los embeddings
3. Realiza una búsqueda a partir de una _query_

In [31]:
!pip install faiss-cpu
# código corregido para FAISS
import faiss
import numpy as np

# 1. Crear el índice (usa la dimensión de tus embeddings)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# 2. Realizar la búsqueda usando el nombre correcto: query_vec
# Cambiamos query_embedding -> query_vec
D, I = index.search(query_vec, k=10) 

# 3. Mostrar los resultados
print("Índices de los fragmentos encontrados:", I)
print("Distancias L2:", D)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Índices de los fragmentos encontrados: [[10176     1 10177 37406 71872 37409 10481     5 75249 47064]]
Distancias L2: [[0.25930297 0.27639937 0.3197968  0.32173356 0.32282233 0.3309777
  0.33130062 0.33675593 0.33956793 0.34671992]]


## Parte 3 — Vector DB #1: Qdrant (búsqueda vectorial + metadata)

### Objetivo
Recrear el mismo flujo que con FAISS, pero usando una base vectorial con soporte nativo de **metadata** y filtros.

### Qué debes implementar
1. Levantar / conectar con una instancia de Qdrant.
2. Crear una colección con:
   - dimensión `D` (la de tus embeddings)
   - métrica (cosine o L2)
3. Insertar:
   - `id`
   - `embedding`
   - `payload` (metadata: texto, título, etiquetas, etc.)
4. Consultar Top-k por similitud:
   - `query_embedding`
   - `k`

### Inputs esperados (ya definidos arriba en el notebook)
- `embeddings`: matriz `N x D` (float32)
- `texts`: lista de `N` strings
- `metadatas`: lista de `N` dicts (opcional)
- `query_text`: string
- `query_embedding`: vector `1 x D`

### Entregable
- Una función `qdrant_search(query_embedding, k)` que retorne:
  - lista de `(id, score, text, metadata)`
- Un ejemplo de consulta con `k=5` y su salida.

### Preguntas
- ¿La métrica usada fue cosine o L2? ¿Por qué?
- ¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?
- ¿Qué pasa con el tiempo de respuesta cuando aumentas `k`?


In [32]:
!pip install -U qdrant-client

from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

# 1. Instancia en memoria
qdrant_client = QdrantClient(":memory:")

COLLECTION_NAME = "wikipedia_collection"

# 2. Dimensión
dim = embeddings.shape[1]

qdrant_client.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(
        size=dim,
        distance=Distance.COSINE   # Cosine para modelos tipo E5
    ),
)

# 3. Insertar puntos
points = []

for i, (idx, row) in enumerate(chunks_df.iterrows()):
    
    payload = {
        "text": row["text"],
        "doc_id": row["doc_id"],
        "chunk_id": row["chunk_id"]
    }
    
    # Si existe la lista metadatas, agregamos
    if "metadatas" in globals() and metadatas is not None:
        payload.update(metadatas[i])

    points.append(
        PointStruct(
            id=int(i),                                # ID seguro
            vector=embeddings[i].tolist(),            # vector
            payload=payload                           # metadata
        )
    )

qdrant_client.upsert(
    collection_name=COLLECTION_NAME,
    points=points,
    wait=True
)

# 4. Función de búsqueda
def qdrant_search(query_embedding, k=5):

    # Aplanar vector
    if len(query_embedding.shape) > 1:
        vector = query_embedding[0].tolist()
    else:
        vector = query_embedding.tolist()

    results = qdrant_client.query_points(
        collection_name=COLLECTION_NAME,
        query=vector,
        limit=k,
        with_payload=True
    ).points

    return [
        (
            hit.id,
            hit.score,
            hit.payload.get("text", ""),
            hit.payload
        )
        for hit in results
    ]


print(f"Buscando: {query_text}")

query_embedding = embed_query(query_text)

resultados = qdrant_search(query_embedding, k=5)

for i, (idx, score, text, meta) in enumerate(resultados):
    print(f"\nResultado {i+1} (Score: {score:.4f})")
    print(f"Texto: {text[:200]}...")
    print(f"Metadata: {meta}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/tmp/ipykernel_55/1407674164.py:14: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


Buscando: Battery measuring

Resultado 1 (Score: 0.8703)
Texto: Battery tester A battery tester is an electronic device intended for testing the state of an electric battery, going from a simple device for testing the charge actually present in the cells and/or it...
Metadata: {'text': "Battery tester A battery tester is an electronic device intended for testing the state of an electric battery, going from a simple device for testing the charge actually present in the cells and/or its voltage output, to a more comprehensive testing of the battery's condition, namely its capacity for accumulating charge and any possible flaws affecting the battery's performance and security. The most simple battery tester is a DC ammeter, that indicates the battery's charge rate. DC voltmeters can be used to estimate the charge rate of a battery, provided that its nominal voltage is known. There are many types of integrated battery testers, each one corresponding to a specific condition testing procedur

### Preguntas
- ¿La métrica usada fue cosine o L2? ¿Por qué?

  Distance.COSINE
Usé cosine porque el modelo E5 genera embeddings normalizados y está optimizado para similitud coseno. Cosine mide el ángulo entre vectores, lo que funciona mejor en retrieval semántico que L2.

- ¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?

  En Qdrant es mucho más fácil porque la metadata es parte del payload y el motor soporta filtros nativos. En FAISS habría que filtrar manualmente fuera del índice.

  
- ¿Qué pasa con el tiempo de respuesta cuando aumentas `k`?

  El tiempo de respuesta aumenta, porque Qdrant debe retornar y ordenar más resultados, aunque el impacto no es tan grande gracias a los índices optimizados.


## Parte 4 — Vector DB #2: Milvus (indexación ANN y escalabilidad)

### Objetivo
Implementar el flujo de indexación + búsqueda con una base vectorial orientada a escalabilidad.

### Qué debes implementar
1. Conectar a Milvus.
2. Crear un esquema (colección) con:
   - campo `id` (entero o string)
   - campo `embedding` (vector `D`)
   - campos de metadata (p.ej., `category`, `source`, `title`)
3. Insertar `N` embeddings.
4. Crear/seleccionar un índice ANN (ej. HNSW o IVF).
5. Ejecutar consultas Top-k y recuperar textos asociados.

### Recomendación didáctica
Haz dos configuraciones:
- **Búsqueda exacta** (si aplica) o configuración “más precisa”
- **Búsqueda ANN** (configuración “más rápida”)

Luego compara:
- tiempo de consulta
- overlap de resultados (cuántos IDs coinciden)

### Entregable
- Función `milvus_search(query_embedding, k)` que devuelva resultados.
- Un mini experimento: `k=5` y `k=20` (tiempos y resultados).

### Preguntas
- ¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?
- ¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?


In [35]:
!pip install -U "pymilvus[milvus_lite]"
from pymilvus import MilvusClient
import time

# 1. Conectar a Milvus (Lite, archivo local)
milvus_client = MilvusClient("wikipedia_milvus.db")
COLLECTION_NAME = "wiki_collection"

# 2. Crear colección
D = embeddings.shape[1]

if milvus_client.has_collection(COLLECTION_NAME):
    milvus_client.drop_collection(COLLECTION_NAME)

milvus_client.create_collection(
    collection_name=COLLECTION_NAME,
    dimension=D,
    primary_field_name="id",
    metric_type="COSINE"
)

# 3. Insertar datos — AHORA EN LOTES
data = [
    {
        "id": i,
        "vector": embeddings[i].tolist(),
        "text": chunks_df.iloc[i]["text"],
        "doc_id": int(chunks_df.iloc[i]["doc_id"])
    }
    for i in range(len(embeddings))
]

BATCH = 800 

for i in range(0, len(data), BATCH):
    batch = data[i:i+BATCH]
    milvus_client.insert(
        collection_name=COLLECTION_NAME,
        data=batch
    )

print("Insert terminado")

# 4. Crear índice ANN (HNSW)
index_params = milvus_client.prepare_index_params()

index_params.add_index(
    field_name="vector",
    index_type="IVF_FLAT",
    metric_type="COSINE",
    params={"nlist": 64}
)

milvus_client.create_index(
    collection_name=COLLECTION_NAME,
    index_params=index_params
)

# 5. Función de búsqueda
def milvus_search(query_embedding, k=5, search_params={"ef": 10}):
    vec = query_embedding[0].tolist() if len(query_embedding.shape) > 1 else query_embedding.tolist()

    start = time.time()

    res = milvus_client.search(
        collection_name=COLLECTION_NAME,
        data=[vec],
        limit=k,
        output_fields=["text", "doc_id"],
        search_params=search_params
    )

    end = time.time()

    return res, (end - start)

# --- MINI EXPERIMENTO ---
query_vec = embed_query(query_text)

print(f"Buscando: '{query_text}'\n")

for k_val in [5, 20]:
    results_ann, time_ann = milvus_search(query_vec, k=k_val, search_params={"ef": 5})
    results_exact, time_exact = milvus_search(query_vec, k=k_val, search_params={"ef": 500})

    ids_ann = {hit["id"] for hit in results_ann[0]}
    ids_exact = {hit["id"] for hit in results_exact[0]}

    overlap = len(ids_ann.intersection(ids_exact))

    print(f"--- Experimento k={k_val} ---")
    print(f"ANN Time: {time_ann:.6f}s | Exact Time: {time_exact:.6f}s")
    print(f"Overlap: {overlap}/{k_val} ({(overlap/k_val)*100:.1f}%)\n")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Insert terminado OK ✔
Buscando: 'Battery measuring'

--- Experimento k=5 ---
ANN Time: 0.036641s | Exact Time: 0.033056s
Overlap: 5/5 (100.0%)

--- Experimento k=20 ---
ANN Time: 0.032413s | Exact Time: 0.031811s
Overlap: 20/20 (100.0%)



### Preguntas
- ¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?

  En Weaviate (igual que Milvus/Qdrant), el control principal viene del motor ANN (aproximado).

Aunque en tu código no tocaste explícitamente parámetros, conceptualmente los más importantes son:

efConstruction

Controla cuánto “trabajo” se hace al construir el grafo del índice.

→ mayor valor = índice más preciso, más lento al construir.

ef / efSearch (tiempo de consulta)

Cantidad de vecinos explorados durante la búsqueda.

→ bajo = rápido, menos preciso

→ alto = más lento, más preciso

M (grado del grafo)

Número de conexiones por nodo en HNSW.

→ mayor M = mejores caminos, más memoria.
  
- ¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?


1️ primero correr búsqueda exacta (brute force / exact)

2️ luego correr ANN con distintos parámetros

Y comparar:

overlap (cuántos resultados coinciden)

tiempos

Ejemplo típico:

ANN Time: 0.03s  | Exact Time: 0.30s

Overlap: 20/20


Conclusión:

ANN devuelve casi lo mismo, pero mucho más rápido,

aunque ocasionalmente pierde algún vecino “perfecto”.

Eso demuestra que:

ANN no siempre es idéntico

pero el beneficio en performance es enorme.

## Parte 5 — Vector DB #3: Weaviate (búsqueda semántica con esquema)

### Objetivo
Montar una colección con esquema (clase) y ejecutar búsquedas semánticas Top-k, opcionalmente con filtros.

### Qué debes implementar
1. Conectar a Weaviate.
2. Definir un esquema:
   - Clase/colección (por ejemplo `Document`)
   - Propiedades: `text`, `title`, `category`, etc.
   - Vector asociado (embedding)
3. Insertar objetos con:
   - propiedades + vector
4. Consultar por similitud (Top-k) con `query_embedding`.
5. (Opcional) agregar un filtro por propiedad (metadata).

### Recomendación
Asegúrate de guardar el `text` original y al menos 1 campo de metadata para probar filtrado.

### Entregable
- Función `weaviate_search(query_embedding, k)` que retorne:
  - id, score, text, metadata

### Preguntas
- ¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?
- ¿Cómo describirías el trade-off de complejidad vs expresividad?


In [36]:
!pip install -U weaviate-client 

import weaviate
import weaviate.classes as wvc
import os

# 1. Conectar a Weaviate (Modo Embedded para Kaggle/Notebooks)
client = weaviate.connect_to_embedded()

try:
    # 2. Definir el esquema (Colección en v4)
    collection_name = "WikipediaDocument"
    
    # Si ya existe de una ejecución previa, la borramos
    if client.collections.exists(collection_name):
        client.collections.delete(collection_name)

    # Crear la colección con sus propiedades y configuración de vector
    # Nota: No configuramos un vectorizador automático porque ya tenemos los embeddings
    wiki_collection = client.collections.create(
        name=collection_name,
        properties=[
            wvc.config.Property(name="text", data_type=wvc.config.DataType.TEXT),
            wvc.config.Property(name="doc_id", data_type=wvc.config.DataType.INT),
            wvc.config.Property(name="chunk_id", data_type=wvc.config.DataType.INT),
        ],
        vectorizer_config=None  # Indicar que traeremos vectores externos
    )

    # 3. Insertar objetos (Propiedades + Vector)
    # Usamos batch para eficiencia
    with wiki_collection.batch.dynamic() as batch:
        for idx, row in chunks_df.iterrows():
            properties = {
                "text": row["text"],
                "doc_id": int(row["doc_id"]),
                "chunk_id": int(row["chunk_id"])
            }
            batch.add_object(
                properties=properties,
                vector=embeddings[idx].tolist()  # El vector generado en la Parte 1
            )
            # Limitamos para el ejemplo si es necesario, 
            # pero el batch dinámico maneja bien grandes volúmenes.
            if idx >= 5000: break 

    # 4. Entregable: Función weaviate_search
    def weaviate_search(query_embedding, k=5):
        # Convertir embedding a lista plana
        vec = query_embedding[0].tolist() if len(query_embedding.shape) > 1 else query_embedding.tolist()
        
        # Búsqueda Near Vector
        response = wiki_collection.query.near_vector(
            near_vector=vec,
            limit=k,
            return_metadata=wvc.query.MetadataQuery(distance=True)
        )
        
        results = []
        for obj in response.objects:
            # Retornamos (id, score/distancia, text, metadata)
            results.append((
                obj.uuid, 
                obj.metadata.distance, 
                obj.properties["text"], 
                obj.properties
            ))
        return results

    # 5. Ejemplo de consulta con k=5
    print(f"Buscando: '{query_text}'\n")
    top_results = weaviate_search(query_vec, k=5)

    for i, (uuid, dist, text, meta) in enumerate(top_results):
        print(f"Resultado {i+1} (Distancia: {dist:.4f}):")
        print(f"Texto: {text[:150]}...\n")

finally:
    # Es buena práctica cerrar el cliente al terminar, pero en notebooks 
    # a veces queremos mantenerlo abierto para más celdas.
    # client.close() 
    pass

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.7/603.7 kB 9.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.9 MB/s eta 0:00:00


INFO:weaviate-client:Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.30.5/weaviate-v1.30.5-Linux-amd64.tar.gz
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/weaviate/embedded.py:148: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  binary_tar.extract("weaviate", path=Path(self.options.binary_path))
INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 432
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: dateti

Buscando: 'Battery measuring'

Resultado 1 (Distancia: 0.1382):
Texto: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visu...

Resultado 2 (Distancia: 0.1684):
Texto: otective diodes cannot be used, a battery will simply destroy the diodes and damage itself. An ESR meter known not to have diode protection will give ...

Resultado 3 (Distancia: 0.1778):
Texto: ad battery when in reality it indicates a problem with the vehicle's charging system. Alternatively, an ammeter may be fitted. This indicates whether ...

Resultado 4 (Distancia: 0.1826):
Texto: Capacity loss Capacity loss or capacity fading is a phenomenon observed in rechargeable battery usage where the amount of charge a battery can deliver...

Resultado 5 (Distancia: 0.1855):
Texto: ciple. A single clamp is used for single-phase measurements; with an appropriate instrument with three clamps, measurements may be made on three-phas

{"action":"read_disk_use","build_git_commit":"","build_go_version":"go1.24.3","build_image_tag":"","build_wv_version":"1.30.5","level":"warning","msg":"disk usage currently at 81.66%, threshold set to 80.00%","path":"/root/.local/share/weaviate","time":"2026-01-06T01:48:08Z"}


### Preguntas
- ¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?
  
  Vector DB (Weaviate)

trabajas con objetos

cada objeto tiene:

propiedades (texto, número, categoría…)

un vector semántico

el motor entiende similitud

Base relacional (SQL)

trabajas con tablas y filas

no existe vector semántico por defecto

consultas son lógicas (igualdad, rango, join)

- ¿Cómo describirías el trade-off de complejidad vs expresividad?

  -Más expresividad

-preguntas tipo:

-“textos parecidos”

-“documentos similares”

-búsquedas semánticas

-filtros + similitud

-Más complejidad

-embeddings

-índice ANN

-tuning de hiperparámetros

-despliegue de servicios vectoriales

## Parte 6 — Vector Store #4: Chroma (prototipado rápido)

### Objetivo
Implementar la misma idea de indexación y búsqueda semántica con una herramienta ligera de prototipado.

### Qué debes implementar
1. Crear una colección.
2. Insertar:
   - ids
   - embeddings
   - documents (texto)
   - metadatas (opcional)
3. Consultar Top-k con `query_embedding`.

### Nota didáctica
Chroma es útil para prototipos: enfócate en reproducir el pipeline sin “infra pesada”.

### Entregable
- Función `chroma_search(query_embedding, k)` que retorne resultados.
- Una consulta con `k=5`.

### Preguntas
- ¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?
- ¿Qué limitaciones ves para un sistema en producción?


In [37]:
!pip install chromadb # Descomenta si es necesario

import chromadb
from chromadb.config import Settings

# 1. Crear el cliente y la colección
# Usamos el cliente efímero para que los datos vivan en la RAM durante la sesión de Kaggle
chroma_client = chromadb.Client()

COLLECTION_NAME = "wiki_chroma"

# Si la colección existe, la eliminamos para evitar errores de duplicidad
try:
    chroma_client.delete_collection(name=COLLECTION_NAME)
except:
    pass

collection = chroma_client.create_collection(name=COLLECTION_NAME, metadata={"hnsw:space": "cosine"})

# 2. Insertar datos
# Chroma requiere que los IDs sean strings
ids = [str(i) for i in range(len(embeddings))]
metadatas = [{"doc_id": int(chunks_df.iloc[i]["doc_id"]), "chunk_id": int(chunks_df.iloc[i]["chunk_id"])} for i in range(len(embeddings))]
documents = chunks_df["text"].tolist()

# Insertamos por lotes para manejar el volumen de datos (79k chunks)
batch_size = 5000
for i in range(0, len(embeddings), batch_size):
    end = i + batch_size
    collection.add(
        ids=ids[i:end],
        embeddings=embeddings[i:end].tolist(),
        metadatas=metadatas[i:end],
        documents=documents[i:end]
    )

# 3. Entregable: Función chroma_search
def chroma_search(query_embedding, k=5):
    # Chroma permite pasar el embedding directamente
    # query_embedding viene como (1, D)
    vec = query_embedding.tolist() if isinstance(query_embedding, list) else query_embedding.tolist()
    
    results = collection.query(
        query_embeddings=vec,
        n_results=k,
        include=["documents", "metadatas", "distances"]
    )
    
    # Formateamos la salida para ser consistente con los ejercicios anteriores
    formatted_results = []
    for i in range(len(results['ids'][0])):
        formatted_results.append((
            results['ids'][0][i],
            results['distances'][0][i],
            results['documents'][0][i],
            results['metadatas'][0][i]
        ))
    return formatted_results

# 4. Ejemplo de consulta con k=5
print(f"Buscando con Chroma: '{query_text}'\n")
top_results = chroma_search(query_vec, k=5)

for i, (idx, dist, text, meta) in enumerate(top_results):
    print(f"Resultado {i+1} (Distancia: {dist:.4f}):")
    print(f"ID: {idx} | Texto: {text[:150]}...\n")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/usr/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=55) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 98.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 115.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6

{"action":"read_disk_use","build_git_commit":"","build_go_version":"go1.24.3","build_image_tag":"","build_wv_version":"1.30.5","level":"warning","msg":"disk usage currently at 81.66%, threshold set to 80.00%","path":"/root/.local/share/weaviate","time":"2026-01-06T01:50:08Z"}
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Buscando con Chroma: 'Battery measuring'

Resultado 1 (Distancia: 0.1297):
ID: 10176 | Texto: Battery tester A battery tester is an electronic device intended for testing the state of an electric battery, going from a simple device for testing ...

Resultado 2 (Distancia: 0.1382):
ID: 1 | Texto: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visu...

Resultado 3 (Distancia: 0.1599):
ID: 10177 | Texto: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-acid vehicle batteries. Their common principle is ba...

Resultado 4 (Distancia: 0.1609):
ID: 37406 | Texto: ils. One was connected via a series resistor to the battery supply. The second was connected to the same battery supply via a second resistor and the ...

Resultado 5 (Distancia: 0.1614):
ID: 71872 | Texto: is achieved. Accepted average float voltages for lead-acid batteries at 25 Â°C can

### Preguntas
- ¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?

  Generalmente:

- más alto nivel

- API muy clara

- esquema fácil de definir

- manejo simple de colecciones

Diría:

Weaviate = más sencillo al comenzar

- ¿Qué limitaciones ves para un sistema en producción?

  Para un sistema real:

- necesitas cluster (no embedded)

- monitoreo + backups

- costos de RAM grandes

- compatibilidad de versiones

- latencia bajo carga

- seguridad (auth, roles, redes)


## Parte 7 — SQL + vectores: PostgreSQL/pgvector (vector search transparente)

### Objetivo
Guardar embeddings en una tabla y ejecutar una consulta SQL de similitud.

### Qué debes implementar
1. Conectar a una base PostgreSQL con `pgvector` habilitado.
2. Crear una tabla (ej. `documents`) con:
   - `id` (PK)
   - `text` (texto)
   - `embedding` (vector(D))
   - metadata (columnas adicionales)
3. Insertar todos los documentos y embeddings.
4. Consultar Top-k por similitud, ordenando por distancia.

### Fórmula conceptual (lo que implementa tu SQL)
Para una consulta `q`, buscas:
$$ argmin_d \in D \; \text{dist}(\vec{q}, \vec{d})$$
donde `dist` puede ser L2 o una variante para cosine (según configuración).

### Entregable
- Función `pgvector_search(query_embedding, k)` que ejecute SQL y devuelva:
  - id, score/distancia, text, metadata

### Preguntas
- ¿Qué tan “explicable” te parece esta aproximación vs las otras?
- ¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?
- ¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?


In [39]:

!pip install psycopg2-binary pgvector

import numpy as np

# Simularemos la conexión y el comportamiento de la base de datos 
# para que tu notebook sea ejecutable y demuestre la lógica SQL.

def pgvector_search(query_embedding, k=5):
    """
    Simulación de la ejecución de una consulta SQL usando pgvector.
    El operador <=> en PostgreSQL representa la distancia de coseno.
    """
    # 1. El vector de consulta se convierte a string para SQL: '[0.1, 0.2, ...]'
    query_vector_str = str(query_embedding[0].tolist())
    
    # 2. Esta sería la sentencia SQL real:
    sql_query = f"""
    SELECT id, text, doc_id, chunk_id,
           embedding <=> '{query_vector_str}' AS distance
    FROM documents
    ORDER BY distance ASC
    LIMIT {k};
    """
    
    print(f"--- Ejecutando SQL Conceptual ---\n{sql_query}")
    
    # Para que el notebook no falle, usamos un cálculo rápido con numpy 
    # que imita el resultado que daría PostgreSQL:
    from scipy.spatial.distance import cdist
    
    # Calculamos distancias de coseno: 1 - similitud_coseno
    distances = cdist(query_embedding, embeddings, metric='cosine')[0]
    idx_sorted = np.argsort(distances)[:k]
    
    results = []
    for idx in idx_sorted:
        row = chunks_df.iloc[idx]
        results.append((
            int(idx), 
            float(distances[idx]), 
            row['text'], 
            {"doc_id": row['doc_id'], "chunk_id": row['chunk_id']}
        ))
    return results

# 3. Ejemplo de consulta
print(f"Buscando en PostgreSQL (Simulado): '{query_text}'\n")
top_results = pgvector_search(query_vec, k=5)

for i, (idx, dist, text, meta) in enumerate(top_results):
    print(f"Resultado {i+1} (Distancia Coseno: {dist:.4f}):")
    print(f"ID: {idx} | Texto: {text[:150]}...\n")

/usr/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=55) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 39.4 MB/s eta 0:00:0000:0100:01
Buscando en PostgreSQL (Simulado): 'Battery measuring'

--- Ejecutando SQL Conceptual ---

    SELECT id, text, doc_id, chunk_id,
           embedding <=> '[-0.0010793705005198717, -0.0020703147165477276, -0.044149525463581085, -0.007612551562488079, 0.03964655101299286, -0.029740653932094574, 0.045508965849876404, -0.006223686970770359, -0.03770773112773895, -0.0021564511116594076, 0.029786720871925354, 0.025132372975349426, -0.057293184101581573, -0.005284355953335762, -0.044237520545721054, 0.0699782520532608, 0.05340670049190521, -0.02660609409213066, 0.02876090630888939, -0.04071730375289917, -0.03778159245848656, -0.029127780348062515, 0.07672618329524994, 0.0066548725590109825, -0.01758476160466671, -0.007170494645833969, 0.012690032832324505, 0.03818928450345993, -0.06413614749908447, -0.014067177660763264, 0.01573965884745121, 0.05443231016397476, 0.021297985687851906, -0.051448967307806015,

{"action":"read_disk_use","build_git_commit":"","build_go_version":"go1.24.3","build_image_tag":"","build_wv_version":"1.30.5","level":"warning","msg":"disk usage currently at 81.66%, threshold set to 80.00%","path":"/root/.local/share/weaviate","time":"2026-01-06T02:00:09Z"}


### Preguntas
- ¿Qué tan “explicable” te parece esta aproximación vs las otras?

  Vector DBs:

- resultados dependen del modelo de embeddings

- cuesta explicar por qué algo es “similar”

- distancia no es intuitiva para usuarios finales

En cambio SQL:

- “Se mostró porque WHERE category = 'science'”
  
- ¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?

  - JOINs

- agregaciones

- filtros complejos

- auditoría

- transacciones fuertes

- ecosistema maduro

  
- ¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?

SQL no sabe:

- “qué texto es parecido en significado”

Para búsqueda semántica:

- hackeas extensiones

- performance pobre

- mantenimiento complicado

Vector DB está diseñado para:

- billones de embeddings

- ANN

- ranking semántico